<a href="https://colab.research.google.com/github/TuanTony5/Pre-AIO-Warm-up-/blob/main/YOLO_Data_Prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Download dataset from kaggle

In [ ]:
# Cài đặt gói kaggle
!pip install kaggle

# Upload file kaggle.json lên Colab
from google.colab import files
files.upload()

# Di chuyển file kaggle.json vào thư mục .kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [2]:
# Tải dữ liệu từ Kaggle bằng cách sử dụng API
!kaggle datasets download -d sshikamaru/car-object-detection

100% 112M/112M [00:08<00:00, 18.3MB/s]
100% 112M/112M [00:08<00:00, 14.6MB/s]


In [ ]:
# Giải nén dữ liệu (nếu cần)
!unzip car-object-detection.zip

# 1. Import libraries

In [4]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.3/78.2 GB disk)


In [5]:
import os
import shutil
import yaml
import numpy as np
import pandas as pd
import cv2
import json
import matplotlib.pyplot as plt

from ultralytics import YOLO
from PIL import Image
from sklearn.model_selection import train_test_split

# 2. Prepare YOLO data

In [6]:
train_data_dir = 'data/training_images'
label_file_path = 'data/train_solution_bounding_boxes (1).csv'
df = pd.read_csv(label_file_path)
df.head()

# YOLO: class x_center y_center width height
# PascalVOC -> YOLO
# (x_min, y_min, x_max, y_max) -> (x_center, y_center, width, height)

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


In [11]:
# Create more columns
df['class'] = 0
df['x_center'] = (df['xmin'] + df['xmax'])/2
df['y_center'] = (df['ymin'] + df['ymax'])/2
df['width'] = df['xmax'] - df['xmin']
df['height'] = df['ymax'] - df['ymin']

df.rename(columns={'image':'img_name'},inplace = True)
df.drop(columns = ['xmin','xmax','ymin','ymax'],inplace = True)
df.head()


,img_name,x_center,y_center,width,height,class
0,vid_4_1000.jpg,304.493488,205.130309,46.468886,36.190476,0
1,vid_4_10000.jpg,67.746744,211.732625,105.166425,49.395109,0
2,vid_4_10040.jpg,300.580318,206.597490,122.775687,59.665380,0
3,vid_4_10020.jpg,563.251809,201.951416,133.536903,59.176319,0
4,vid_4_10060.jpg,74.594790,212.466216,115.927641,51.840412,0
...,...,...,...,...,...,...
554,vid_4_9860.jpg,24.617626,217.272506,49.235251,37.901554,0
555,vid_4_9880.jpg,433.270211,203.490123,206.788055,94.015544,0
556,vid_4_9900.jpg,70.898762,203.736237,141.797524,70.880829,0
557,vid_4_9960.jpg,552.173343,200.536755,129.488711,56.606218,0
